In [1]:
import numpy as np
import itertools
from environment.env import GridEnvDeform
from tqdm import trange

In [2]:
# maze size
N = 2

# thetas deformations (range(a,b),range(c,d))
l0 = 1
h0 = 10
l1 = 1
h1 = 10

maze = np.load(f"maze/maze_{N}.npy")

In [3]:
from stable_baselines3 import DQN
from environment.env import MDPGYMGridEnvDeform

In [6]:

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder

def make_env():
    N = 2

    # thetas deformations (range(a,b),range(c,d))
    l0 = 1
    h0 = 10
    l1 = 1
    h1 = 10
    
    maze = np.load(f"maze/maze_{N}.npy")
    env = MDPGYMGridEnvDeform(maze,l0,h0,l1,h1, render_mode="human")

    env = Monitor(env)  # record stats such as returns
    return env

env = DummyVecEnv([make_env])

model= DQN.load("agents/pretrained/MDP/DQNsb3_z427ps8r.zip", print_system_info=True)



== CURRENT SYSTEM INFO ==
- OS: Linux-6.11.2-amd64-x86_64-with-glibc2.40 # 1 SMP PREEMPT_DYNAMIC Kali 6.11.2-1kali1 (2024-10-15)
- Python: 3.9.19
- Stable-Baselines3: 2.3.2
- PyTorch: 2.4.1+cu121
- GPU Enabled: True
- Numpy: 2.0.2
- Cloudpickle: 3.0.0
- Gymnasium: 0.29.1
- OpenAI Gym: 0.26.2

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-6.10.3-200.fc40.x86_64-x86_64-with-glibc2.39 # 1 SMP PREEMPT_DYNAMIC Mon Aug  5 14:30:00 UTC 2024
- Python: 3.11.10
- Stable-Baselines3: 2.3.2
- PyTorch: 2.5.1+cu121
- GPU Enabled: False
- Numpy: 1.26.3
- Cloudpickle: 3.1.0
- Gymnasium: 0.29.1



/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(


In [ ]:
def eval_agent_mdp(agent,env,num_episodes,max_episode_steps,render):
    """Returns
        - episode_transition: list of list of tuples (s,a,r,s',done), t[i] is the ith episode
        - beliefs: list of beliefs at each time step 
    """
    
    # if render:
    #     env.set_rendering()

    for i in trange(num_episodes):

        s = env.reset()

        totalReward = 0.0
        done = False
        steps = 0
        episode_transitions = []

        while not done and steps < max_episode_steps:

            best_action, _ = agent.predict(s,deterministic=True)

            next_state, reward, done, info = env.step(best_action)

            totalReward += reward            

            if render:
                # print("State", s)
                # print("Action: ", best_action)
                # print("Reward:     " + str(totalReward) + "  ")
                # print("Next State: ", next_state)
                # print("\n")
            
                env.render()

            s = next_state
            steps += 1

eval_agent_mdp(model,env,10,100,True)

100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


: 

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

In [7]:
mean_reward, std_reward

(np.float64(-196.85), np.float64(4.95))